In [1]:
import sqlite3
import pandas as pd

# 1.1 Movies Database

Take the movies dataset and turn it into a single `sqlite` database. It should have one table for each csv file in the movies dataset

In [2]:
credits_df = pd.read_csv(r'C:\Users\David\Documents\code\archive\credits.csv')
keywords_df = pd.read_csv(r'C:\Users\David\Documents\code\archive\keywords.csv')
linkss_df = pd.read_csv(r'C:\Users\David\Documents\code\archive\links_small.csv')
meta_df = pd.read_csv(r'C:\Users\David\Documents\code\archive\movies_metadata.csv')
ratingss_df = pd.read_csv(r'C:\Users\David\Documents\code\archive\ratings_small.csv')
ratings_df = pd.read_csv(r'C:\Users\David\Documents\code\archive\ratings.csv')

conn = sqlite3.connect('Kaggle_Movies')
c = conn.cursor()

credits_df.to_sql(
    name="credits",
    con=conn, 
    schema=None, 
    if_exists='replace', 
    index=True, 
)
keywords_df.to_sql(
    name="keywords",
    con=conn, 
    schema=None, 
    if_exists='replace', 
    index=True, 
)
linkss_df.to_sql(
    name="links_small",
    con=conn, 
    schema=None, 
    if_exists='replace', 
    index=True, 
)
meta_df.to_sql(
    name="meta",
    con=conn, 
    schema=None, 
    if_exists='replace', 
    index=True, 
)
ratingss_df.to_sql(
    name="ratings_small",
    con=conn, 
    schema=None, 
    if_exists='replace', 
    index=True, 
)
ratings_df.to_sql(
    name="ratings",
    con=conn, 
    schema=None, 
    if_exists='replace', 
    index=True, 
)

# 1.2 Queries

**1.2.1** Use a single query to pull the original title of movies with a budget above $5m

**1.2.2** Use a query to pull the english-language films with the word `war` in their title

**1.2.3** Left join the average ratings from the `ratings` table onto the `movies_metadata` table, so you can have a relation between budget and rating. Hint: use a subquery.

In [ ]:
#1.2.1
pd.read_sql(
"""
SELECT *
FROM meta
WHERE budget > 5000000
""", 
    con=conn
)

In [ ]:
#1.2.2
pd.read_sql(
"""
SELECT *
FROM meta
WHERE original_language = 'en'
    and original_title LIKE '%war%'
""", 
    con=conn
)

In [ ]:
#1.2.3

pd.read_sql(
"""
SELECT *
FROM meta 
LEFT JOIN (

SELECT movieId, 
AVG (rating) 
FROM ratings 
GROUP BY movieId

)
    ON meta.id = movieId 
""", con=conn)

# 2. Baseball Database

The [Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/) has an sqlite version. Download it for these exercises.

**2.1** Which player has had the most homeruns?

**2.2** Is there a relation between how many homeruns a player has made in a year and his salary that year? Pull both colums together in a single query



In [53]:
conn = sqlite3.connect(r'C:\Users\David\Documents\code\lahmansbaseballdb.sqlite')

# 2.1
pd.read_sql(
"""
SELECT nameFirst, nameLast, MAX(HR)
FROM (
SELECT *
FROM batting 
LEFT JOIN people
    ON batting.playerID = people.playerID 
    )
""", con=conn)



,nameFirst,nameLast,MAX(HR)
0,Barry,Bonds,73


In [52]:
# 2.2
pd.read_sql(
"""
SELECT playerID, HR, salary
FROM (
SELECT *
FROM batting 
LEFT JOIN salaries
    ON batting.playerID = salaries.playerID 
    )
""", con=conn)

,playerID,HR,salary
0,abercda01,0,NaN
1,addybo01,0,NaN
2,allisar01,0,NaN
3,allisdo01,2,NaN
4,ansonca01,0,NaN
...,...,...,...
382156,zobribe01,1,7000000.0
382157,zobribe01,1,7500000.0
382158,zobribe01,1,10500000.0
382159,zuninmi01,9,504100.0


In [55]:
# 2.2 (continue'd)
# To actually answer the question
df = pd.read_sql(
"""
SELECT playerID, HR, salary
FROM (
SELECT *
FROM batting 
LEFT JOIN salaries
    ON batting.playerID = salaries.playerID 
    )
""", con=conn)

df.corr()
#Somewhat, yes.

,HR,salary
HR,1.000000,0.260229
salary,0.260229,1.000000
